In [86]:
import pandas as pd
from sklearn.metrics import ndcg_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, average_precision_score
import matplotlib.pyplot as plt


train_df = pd.read_csv('train/train_df.csv')
test_df = pd.read_csv('test/test_df.csv')
print("ok")

ok


In [87]:
X_train = train_df.drop(['search_id', 'target'], axis=1)
y_train = train_df['target']
X_test = test_df.drop(['search_id', 'target'], axis=1)
y_test = test_df['target']
print("ok")

ok


In [88]:

# Устанавливаем sampling_strategy так, чтобы минорных примеров было в 5 раз меньше мажорных
ros = RandomOverSampler(sampling_strategy=0.2, random_state=0)

# Применяем увеличение выборки к обучающим данным
# X_train, y_train = ros.fit_resample(X_train, y_train)


In [89]:


def evaluate(model, testing_set_x, testing_set_y):
    # Получение предсказанных вероятностей для положительного класса
    predictions_proba = model.predict(testing_set_x)
    
    # Вычисление метрик
    accuracy = accuracy_score(testing_set_y, predictions_proba >= 0.5)
    roc_auc = roc_auc_score(testing_set_y, predictions_proba)
    precision = precision_score(testing_set_y, predictions_proba >= 0.5)
    recall = recall_score(testing_set_y, predictions_proba >= 0.5)
    pr_auc = average_precision_score(testing_set_y, predictions_proba)
    
    # Формирование результата
    result = pd.DataFrame([[accuracy, precision, recall, roc_auc, pr_auc]], columns=['Accuracy', 'Precision', 'Recall', 'ROC_auc', 'PR_auc'])
    return result

In [90]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


params = {
    'objective': 'binary',
    'metric': 'auc',  # Используем AUC как основную метрику
    'verbose': -1,
    'scale_pos_weight': 50,
}

evals_result = {}  # Словарь для сохранения результатов

bst = lgb.train(params, train_data, valid_sets=[test_data], callbacks=[lgb.record_evaluation(evals_result), lgb.early_stopping(stopping_rounds=10)])
print(evals_result)
# Вызов функции evaluate для вычисления метрик на тестовом наборе данных
results = evaluate(bst, X_test, y_test)
print(results)

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.747433
{'valid_0': OrderedDict([('auc', [0.7384221916191226, 0.7474326185323628, 0.742799527837891, 0.7349596694865237, 0.7234015345268542, 0.7178339563250049, 0.7206177454259296, 0.7118433995671848, 0.7190537084398977, 0.7137615581349597, 0.714027149321267, 0.7141943734015346])])}
   Accuracy  Precision    Recall   ROC_auc   PR_auc
0   0.95291   0.193548  0.352941  0.747433  0.13232


In [91]:
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)


In [92]:
# Мы могли бы группировать данные по search_id и рассчитывать NDCG для каждой группы,
# но поскольку задача стоит рассчитать NDCG для всех документов, мы рассчитаем её глобально.
ndcg_score = ndcg_score([y_test], [y_pred], k=None)
print(f'NDCG Score: {ndcg_score}')


NDCG Score: 0.5354627311464872
